# Master MIDI Dataset Search and Filter (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2023

***

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/tegridy-tools
!pip install huggingface_hub
!pip install tqdm

In [ ]:
#@title Import all needed modules

print('Loading core modules... Please wait...')
import os
import copy
from collections import Counter
import random
import pickle
from tqdm import tqdm
import pprint
import statistics
import shutil

print('Creating IO dirs... Please wait...')

if not os.path.exists('/content/Main-MIDI-Dataset'):
    os.makedirs('/content/Main-MIDI-Dataset')

if not os.path.exists('/content/Master-MIDI-Dataset'):
    os.makedirs('/content/Master-MIDI-Dataset')

if not os.path.exists('/content/Output-MIDI-Dataset'):
    os.makedirs('/content/Output-MIDI-Dataset')

print('Loading TMIDIX module...')
os.chdir('/content/tegridy-tools/tegridy-tools')

import TMIDIX

print('Done!')

from huggingface_hub import hf_hub_download

os.chdir('/content/')
print('Enjoy! :)')

# (PREP MAIN MIDI DATASET)

In [ ]:
#@title Download main MIDI dataset
print('=' * 70)
print('Downloading Los Angeles MIDI Dataset...Please wait...')
print('=' * 70)

hf_hub_download(repo_id='projectlosangeles/Los-Angeles-MIDI-Dataset', 
                filename='Los-Angeles-MIDI-Dataset-Ver-2-0-CC-BY-NC-SA.zip',
                repo_type="dataset",
                local_dir='/content/Main-MIDI-Dataset', 
                local_dir_use_symlinks=False)
print('=' * 70)
print('Done! Enjoy! :)')
print('=' * 70)

In [ ]:
#@title Unzip main MIDI dataset
%cd /content/Main-MIDI-Dataset/

print('=' * 70)
print('Unzipping Los Angeles MIDI Dataset...Please wait...')
!unzip 'Los-Angeles-MIDI-Dataset-Ver-2-0-CC-BY-NC-SA.zip'
print('=' * 70)

print('Done! Enjoy! :)')
print('=' * 70)
%cd /content/

In [ ]:
#@title Create main MIDI dataset files list
print('=' * 70)
print('Creating dataset files list...')
dataset_addr = "/content/Main-MIDI-Dataset/MIDIs"

# os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]

if filez == []:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

print('=' * 70)
print('Randomizing file list...')
random.shuffle(filez)
print('=' * 70)

LAMD_files_list = []

for f in tqdm(filez):
  LAMD_files_list.append([f.split('/')[-1].split('.mid')[0], f])
print('Done!')
print('=' * 70)

In [ ]:
#@title Load main MIDI dataset metadata
print('=' * 70)
print('Loading LAMDa data...Please wait...')
print('=' * 70)
print('Loading LAMDa META-DATA...')
meta_data = pickle.load(open('/content/Main-MIDI-Dataset/META_DATA/LAMDa_META_DATA.pickle', 'rb'))
print('Done!')

# (SEARCH AND FILTER)

### DO NOT FORGET TO UPLOAD YOUR MASTER DATASET TO "Master-MIDI-Dataset" FOLDER

In [ ]:
#@title Master MIDI Dataset Search and Filter

#@markdown NOTE: You can stop the search at any time to render partial results

number_of_top_ratios_MIDIs_to_collect = 10 #@param {type:"slider", min:1, max:20, step:1}

#@markdown Match ratio control option

maximum_match_ratio_to_search_for = 1 #@param {type:"slider", min:0, max:1, step:0.01}

#@markdown MIDI pitches search options

pitches_counts_cutoff_threshold_ratio = 0 #@param {type:"slider", min:0, max:1, step:0.05}
search_transposed_pitches = False #@param {type:"boolean"}
skip_exact_matches = False #@param {type:"boolean"}

#@markdown Additional search options

add_pitches_counts_ratios = True #@param {type:"boolean"}
add_timings_ratios = False #@param {type:"boolean"}
add_durations_ratios = False #@param {type:"boolean"}

print('=' * 70)
print('Master MIDI Dataset Search and Filter')
print('=' * 70)

###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Master-MIDI-Dataset"
# os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

if filez == []:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

print('Randomizing file list...')
random.shuffle(filez)
print('=' * 70)
###################

input_files_count = 0
files_count = 0

for f in filez:
    try:
        
        input_files_count += 1

        fn = os.path.basename(f)
        fn1 = fn.split('.mid')[0]
        ext = fn.split('.')[-1]

        if ext == 'mid' or ext == 'midi' or ext == 'kar':

          print('Processing MIDI File #', files_count+1, 'out of', len(filez))
          print('MIDI file name', fn)
          print('-' * 70)          

          #=======================================================

          score = TMIDIX.midi2ms_score(open(f, 'rb').read())

          events_matrix = []

          itrack = 1

          while itrack < len(score):
              for event in score[itrack]:         
                events_matrix.append(event)
              itrack += 1

          # Sorting...
          events_matrix.sort(key=lambda x: x[1])

          # recalculating timings
          for e in events_matrix:
              e[1] = int(e[1] / 10)
              if e[0] == 'note':
                e[2] = int(e[2] / 20)

          # final processing...

          melody_chords = []

          patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

          pe = events_matrix[0]
          for e in events_matrix:

            if e[0] == 'note':
              # ['note', start_time, duration, channel, note, velocity]
              time = max(0, min(255, e[1]-pe[1]))
              duration = max(1, min(255, e[2]))
              channel = max(0, min(15, e[3]))

              if e[3] != 9:
                instrument = max(0, min(127, patches[e[3]]))
              else:
                instrument = max(128, min(255, patches[e[3]]+128))

              if e[3] != 9:

                pitch = max(1, min(127, e[4]))
              else:
                pitch = max(129, min(255, e[4]+128))

              if e[3] != 9:
                velocity = max(1, min(127, e[5]))
              else:
                velocity = max(129, min(255, e[5]+128))

              melody_chords.append([time, duration, channel, instrument, pitch, velocity])

            if e[0] == 'patch_change':
              # ['patch_change', dtime, channel, patch]
              time = max(0, min(127, e[1]-pe[1]))
              channel = max(0, min(15, e[2]))
              patch = max(0, min(127, e[3]))

              patches[channel] = patch

            pe = e # Previous event

          MATRIX = [[0]*256 for i in range(38)]

          for m in melody_chords:

            MATRIX[0][m[0]] += 1
            MATRIX[1][m[1]] += 1
            MATRIX[2][m[2]] += 1 
            MATRIX[3][m[3]] += 1
            MATRIX[4][m[4]] += 1
            MATRIX[5][m[5]] += 1
            MATRIX[m[2]+6][m[3]] += 1
            MATRIX[m[2]+22][m[4]] += 1

          #==================================================

          score = TMIDIX.midi2score(open(f, 'rb').read())

          events_matrix = []

          track_count = 0

          for s in score:
              
              if track_count > 0:
                  track = s
                  track.sort(key=lambda x: x[1])
                  events_matrix.extend(track)
              else:
                  midi_ticks = s

              track_count += 1
              
          events_matrix.sort(key=lambda x: x[1])

          mult_pitches_counts = []

          for i in range(-6, 6):

            events_matrix1 = []

            for e in events_matrix:

              ev = copy.deepcopy(e)

              if e[0] == 'note':
                  if e[3] == 9:
                      ev[4] = ((e[4] % 128) + 128)
                  else:
                    ev[4] = ((e[4] % 128) + i)

                  events_matrix1.append(ev)

            pitches_counts = [[y[0],y[1]] for y in Counter([y[4] for y in events_matrix1 if y[0] == 'note']).most_common()]
            pitches_counts.sort(key=lambda x: x[0], reverse=True)
            
            mult_pitches_counts.append(pitches_counts)

          patches_list = sorted(list(set([y[3] for y in events_matrix if y[0] == 'patch_change'])))

          #==================================================

          ms_score = TMIDIX.midi2ms_score(open(f, 'rb').read())

          ms_events_matrix = []

          itrack1 = 1

          while itrack1 < len(ms_score):
              for event in ms_score[itrack1]:         
                  if event[0] == 'note':
                      ms_events_matrix.append(event)
              itrack1 += 1

          ms_events_matrix.sort(key=lambda x: x[1])


          chords = []
          pe = ms_events_matrix[0]
          cho = []
          for e in ms_events_matrix:
              if (e[1] - pe[1]) == 0:
                if e[3] != 9:
                  if (e[4] % 12) not in cho:
                    cho.append(e[4] % 12)
              else:
                if len(cho) > 0:
                  chords.append(sorted(cho))
                cho = []
                if e[3] != 9:
                  if (e[4] % 12) not in cho:
                    cho.append(e[4] % 12)

              pe = e
              
          if len(cho) > 0:
              chords.append(sorted(cho))

          ms_chords_counts = sorted([[list(key), val] for key,val in Counter([tuple(c) for c in chords if len(c) > 1]).most_common()], reverse=True, key = lambda x: x[1])

          times = []
          pt = ms_events_matrix[0][1]
          start = True
          for e in ms_events_matrix:
              if (e[1]-pt) != 0 or start == True:
                  times.append((e[1]-pt))
                  start = False
              pt = e[1]
              
          durs = [e[2] for e in ms_events_matrix]
          vels = [e[5] for e in ms_events_matrix]

          avg_time = int(sum(times) / len(times))
          avg_dur = int(sum(durs) / len(durs))

          mode_time = statistics.mode(times)
          mode_dur = statistics.mode(durs)

          median_time = int(statistics.median(times))
          median_dur = int(statistics.median(durs))

          #=======================================================

          print('Searching for matches...Please wait...')
          print('-' * 70)

          final_ratios = []

          for d in tqdm(meta_data):

              p_counts = d[1][10][1]
              p_counts.sort(reverse = True, key = lambda x: x[1])
              max_p_count = p_counts[0][1]
              trimmed_p_counts = [y for y in p_counts if y[1] >= (max_p_count * pitches_counts_cutoff_threshold_ratio)]
              total_p_counts = sum([y[1] for y in trimmed_p_counts])
              
              if search_transposed_pitches:
                search_pitches = mult_pitches_counts
              else:
                search_pitches = [mult_pitches_counts[6]]

              #===================================================

              ratios_list = []

              #===================================================

              atrat = [0]

              if add_timings_ratios:

                source_times = [avg_time, 
                                median_time, 
                                mode_time]

                match_times = meta_data[0][1][3][1]

                times_ratios = []

                for i in range(len(source_times)):
                  maxtratio = max(source_times[i], match_times[i])
                  mintratio = min(source_times[i], match_times[i])
                  times_ratios.append(mintratio / maxtratio)

                avg_times_ratio = sum(times_ratios) / len(times_ratios)

                atrat[0] = avg_times_ratio

              #===================================================

              adrat = [0]

              if add_durations_ratios:

                source_durs = [avg_dur,
                                median_dur,
                                mode_dur]

                match_durs = meta_data[0][1][4][1]

                durs_ratios = []

                for i in range(len(source_durs)):
                  maxtratio = max(source_durs[i], match_durs[i])
                  mintratio = min(source_durs[i], match_durs[i])
                  durs_ratios.append(mintratio / maxtratio)

                avg_durs_ratio = sum(durs_ratios) / len(durs_ratios)

                adrat[0] = avg_durs_ratio

              #===================================================

              for m in search_pitches:

                sprat = []

                m.sort(reverse = True, key = lambda x: x[1])
                max_pitches_count = m[0][1]
                trimmed_pitches_counts = [y for y in m if y[1] >= (max_pitches_count * pitches_counts_cutoff_threshold_ratio)]
                total_pitches_counts = sum([y[1] for y in trimmed_pitches_counts])

                same_pitches = set([T[0] for T in trimmed_p_counts]) & set([m[0] for m in trimmed_pitches_counts])
                num_same_pitches = len(same_pitches)
                same_pitches_ratio = (num_same_pitches / len(set([m[0] for m in trimmed_p_counts]+[T[0] for T in trimmed_pitches_counts])))

                if skip_exact_matches:
                  if same_pitches_ratio == 1:
                    same_pitches_ratio = 0

                sprat.append(same_pitches_ratio)

                #===================================================

                spcrat = [0]

                if add_pitches_counts_ratios:

                  same_trimmed_p_counts = sorted([T for T in trimmed_p_counts if T[0] in same_pitches], reverse = True)
                  same_trimmed_pitches_counts = sorted([T for T in trimmed_pitches_counts if T[0] in same_pitches], reverse = True)

                  same_trimmed_p_counts_ratios = [[s[0], s[1] / total_p_counts] for s in same_trimmed_p_counts]
                  same_trimmed_pitches_counts_ratios = [[s[0], s[1] / total_pitches_counts] for s in same_trimmed_pitches_counts]

                  same_pitches_counts_ratios = []

                  for i in range(len(same_trimmed_p_counts_ratios)):
                    mincratio = min(same_trimmed_p_counts_ratios[i][1], same_trimmed_pitches_counts_ratios[i][1])
                    maxcratio = max(same_trimmed_p_counts_ratios[i][1], same_trimmed_pitches_counts_ratios[i][1])
                    same_pitches_counts_ratios.append([same_trimmed_p_counts_ratios[i][0], mincratio / maxcratio])

                  same_counts_ratios = [s[1] for s in same_pitches_counts_ratios]

                  if len(same_counts_ratios) > 0:
                    avg_same_pitches_counts_ratio = sum(same_counts_ratios) / len(same_counts_ratios)
                  else:
                    avg_same_pitches_counts_ratio = 0

                  spcrat[0] = avg_same_pitches_counts_ratio

                #===================================================

                r_list = [sprat[0]]

                if add_pitches_counts_ratios:
                  r_list.append(spcrat[0])
                
                if add_timings_ratios:
                  r_list.append(atrat[0])
                  
                if add_durations_ratios:
                  r_list.append(adrat[0])

                ratios_list.append(r_list)

              #===================================================
              
              avg_ratios_list = []

              for r in ratios_list:
                avg_ratios_list.append(sum(r) / len(r))

              #===================================================
              
              final_ratio = max(avg_ratios_list)
              
              if final_ratio > maximum_match_ratio_to_search_for:
                  final_ratio = 0

              final_ratios.append(final_ratio)

          #=======================================================

          print('-' * 70)

          max_ratios = sorted(final_ratios, reverse=True)[:number_of_top_ratios_MIDIs_to_collect]

          print('Max match ratio', max_ratios[0])
          print('-' * 70)
          print('Copying max ratios MIDIs...')

          for i in range(number_of_top_ratios_MIDIs_to_collect):

            max_ratio = max_ratios[i]
            max_ratio_index = final_ratios.index(max_ratio)

            ffn = meta_data[max_ratio_index][0]
            ffn_idx = [y[0] for y in LAMD_files_list].index(ffn)

            ff = LAMD_files_list[ffn_idx][1]

            #=======================================================
            
            dir_str = str(fn1)
            copy_path = '/content/Output-MIDI-Dataset/'+dir_str
            if not os.path.exists(copy_path):
                os.mkdir(copy_path)

            shutil.copy2(f, copy_path+'/'+fn)

            fff = str(max_ratio * 100) + '_' + ffn

            shutil.copy2(ff, copy_path+'/'+fff)
          
          #=======================================================
          
          print('Done!')
          print('=' * 70)

          #=======================================================

          # Processed files counter
          files_count += 1
                                        
    except KeyboardInterrupt:
        print('Quitting...')
        print('Total number of processed MIDI files', files_count)
        print('=' * 70)
        break  

    except Exception as ex:
        print('WARNING !!!')
        print('=' * 70)
        print('Bad file:', f)
        print('Error detected:', ex)
        print('=' * 70)
        continue

print('Total number of processed MIDI files', files_count)
print('=' * 70)

# Congrats! You did it! :)